In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.regularizers import l2
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D
from keras.applications import InceptionV3, Xception, ResNet152V2, InceptionResNetV2, DenseNet201, ResNet50

In [ ]:
base_model= InceptionV3(include_top=False, weights="imagenet", input_shape=(299,299,3))
base_model= InceptionResNetV2(include_top=False, weights="imagenet", input_shape=(299,299,3))
# base_model= DenseNet201(include_top=False, weights="imagenet", input_shape=(299,299,3))

In [ ]:
base_model.trainable = False

In [ ]:
# # nb_classes = 7
# model = Sequential()
# model.add(base_model)
# model.add(GlobalAveragePooling2D())
# model.add(Dense(7, activation='relu'))
# model.summary()

In [ ]:

model= Sequential()
model.add(base_model)
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
# model.add(GlobalAveragePooling2D())
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))
model.summary()

In [ ]:
# 
data=pd.read_csv("HAM10000_metadata_kaggle.csv")
# Quitar duplicados
data.drop_duplicates(subset='lesion_id', keep="first")
data['image_full_name']=data['image_id']+'.jpg'
X=data[['image_full_name','dx','lesion_id']]

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
Y=X.pop('dx').to_frame()
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
Y.head()

In [ ]:
train=pd.concat([X_train,y_train],axis=1)
val=pd.concat([X_val,y_val],axis=1)
test=pd.concat([X_test,y_test],axis=1)

In [ ]:
print(X_train)
print(y_train)

In [ ]:

from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
encoder.fit(val['dx'])
name_as_indexes_train=encoder.transform(val['dx']) 
val['label']=name_as_indexes_train


In [ ]:
encoder=LabelEncoder()
encoder.fit(test['dx'])
name_as_indexes_test=encoder.transform(test['dx']) 
test['label']=name_as_indexes_test


In [ ]:
from keras_preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale = 1./255,
                                     rotation_range=10,  
                                     zoom_range = 0.1, 
                                     width_shift_range=0.0,  height_shift_range=0.0) 

In [ ]:
train_data= train_generator.flow_from_dataframe(dataframe=train,x_col="image_full_name",y_col="dx",
                                                batch_size=64,directory="ISIC2018_Task3_Training_Input",
                                                shuffle=True,class_mode="categorical",target_size=(299,299))

In [ ]:
train_data.class_indices


In [ ]:

test_generator=ImageDataGenerator(rescale = 1./255)

In [ ]:
test_data= test_generator.flow_from_dataframe(dataframe=test,x_col="image_full_name",y_col="dx",
                                              directory="ISIC2018_Task3_Training_Input",
                                              shuffle=False,batch_size=1,class_mode=None,target_size=(299,299))

In [ ]:
val_data=test_generator.flow_from_dataframe(dataframe=val,x_col="image_full_name",y_col="dx",
                                            directory="ISIC2018_Task3_Training_Input",
                                            batch_size=64,shuffle=False,class_mode="categorical",target_size=(299,299))

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_control = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=.5, min_lr=0.00001)

In [ ]:
from keras.callbacks import ModelCheckpoint
# Save the model with best weights
checkpointer = ModelCheckpoint('modelazo.hdf5', verbose=1,save_best_only=True, monitor='val_accuracy',mode='max')

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='max',restore_best_weights=True)

In [ ]:
adam = optimizers.Adam(lr=0.0001)
model.compile(optimizer=adam, loss="categorical_crossentropy",
              metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
history = model.fit(train_data,
                    steps_per_epoch=train_data.samples//train_data.batch_size,
                    validation_data=val_data,
                    verbose=1,
                    validation_steps=val_data.samples//val_data.batch_size,
                    epochs=100, callbacks=[learning_control, checkpointer, early_stopping])

In [ ]:
model.save('modelazo.hdf5')

In [ ]:
test_data.reset()
predictions = model.predict_generator(test_data, steps=test_data.samples/test_data.batch_size,verbose=1)
y_pred= np.argmax(predictions, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix 
cm= confusion_matrix(name_as_indexes_test,y_pred)
print(cm)

In [ ]:
history.history['loss']

In [ ]:
# Función para graficar la historia
import numpy as np
import matplotlib.pyplot as plt
EPOCAS = 43

def graficar(h):
    LOSS = 0
    ACCURACY = 1
    entrenamiento = np.zeros((2, EPOCAS))
    prueba = np.zeros((2, EPOCAS))
    entrenamiento[LOSS] = h.history['loss']  # loss de entrenamiento
    prueba[LOSS] = h.history['val_loss']    # loss de validación
    entrenamiento[ACCURACY] = h.history['accuracy']  # acc de entrenamiento
    prueba[ACCURACY] = h.history['val_accuracy']  # acc de validación
    epocas = range(1, EPOCAS+1)
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    for i, label in zip((LOSS, ACCURACY), ('perdida', 'exactitud')):
        axs[i].plot(epocas, entrenamiento[i], 'b-', label='Entrenamiento '+label)
        axs[i].plot(epocas, prueba[i], 'y-', label='Prueba '+label)
        axs[i].set_title('Entrenamiento y prueba ' + label)
        axs[i].set_xlabel('Epocas')
        axs[i].set_ylabel(label)
        axs[i].legend()
    plt.show()

graficar(history)

### Validación Cruzada

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
X.shape[0]

In [ ]:
model= Sequential()
model.add(base_model)
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
# model.add(GlobalAveragePooling2D())
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))
model.summary()

adam = optimizers.Adam(lr=0.0001)
model.compile(optimizer=adam, loss="categorical_crossentropy",
              metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
history = model.fit(train_data,
                    steps_per_epoch=train_data.samples//train_data.batch_size,
                    validation_data=val_data,
                    verbose=1,
                    validation_steps=val_data.samples//val_data.batch_size,
                    epochs=100, callbacks=[learning_control, checkpointer, early_stopping])

In [ ]:
X

In [ ]:

X.to_numpy()

In [ ]:
kfold = StratifiedKFold(n_splits=2, shuffle=True)
for entr, prue in kfold.split(X, np.zeros(shape=(X.shape[0], 1))):
    X.to_numpy()[entr]
    

In [ ]:
X_cross = X.to_numpy()
Y_cross = Y.to_numpy()

In [ ]:
data_cross = data.to_numpy()
# data_cross = data_cross.reshape(data_cross.shape[0], 299, 299, 3)
data

In [ ]:
X_cross[]

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Definir una función que crea y compila el modelo
def create_model():
    modelazo = Sequential()
    # Agregar capas al modelo
    modelazo.add(model)
    modelazo.compile(optimizer='adam',
                     loss='categorical_crossentropy', metrics=['accuracy'])
    return modelazo

# Crear un clasificador de Keras
modelCross = KerasClassifier(build_fn=create_model, epochs=1, batch_size=64)

# Obtener todos los datos de entrenamiento
# X_cross_train = []
# y_cross_train = []
# for i in range(len(train_data)):
#     batch = next(train_data)
#     X_batch = batch[0]
#     y_batch = batch[1]
#     X_cross_train.append(X_batch)
#     y_cross_train.append(y_batch)
# X_cross_train = np.concatenate(X_cross_train)
# y_cross_train = np.concatenate(y_cross_train)

# Realizar la validación cruzada
scores = cross_val_score(modelCross, X=X_train, y=y_train, cv=2)
print(scores)